<a href="https://colab.research.google.com/github/cedro3/stylegan2/blob/master/Edit_new_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Edit_new_image

---



# 1.セットアップ
指定環境である **tensorflow 1.14** に変更し、それに伴い **keras 2.0.8** に変更します。そして、githubからコードをダウンロードし、ディレクトリをstylegan2に移します。


In [ ]:
!pip install tensorflow==1.14
!pip install tensorflow-gpu==1.14
!pip install keras==2.0.8
!git clone https://github.com/cedro3/stylegan2.git

In [ ]:
cd stylegan2

必要な関数を定義します。


In [ ]:
import numpy as np
import scipy.ndimage
import os
import PIL.Image
import sys
import bz2
from keras.utils import get_file
import dlib
import argparse
import numpy as np
import dnnlib
import dnnlib.tflib as tflib
import re
import projector
import pretrained_networks
from training import dataset
from training import misc
import matplotlib.pyplot as plt
from tqdm import trange

# -------------- 潜在変数によって生成する顔画像の表示 -------------
def display(vec_syn):
  
    network_pkl = 'gdrive:networks/stylegan2-ffhq-config-f.pkl'  
    
    _G, _D, Gs = pretrained_networks.load_networks(network_pkl)
    noise_vars = [var for name, var in Gs.components.synthesis.vars.items() if name.startswith('noise')]

    Gs_syn_kwargs = dnnlib.EasyDict()
    Gs_syn_kwargs.output_transform = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
    Gs_syn_kwargs.randomize_noise = True 
    Gs_syn_kwargs.truncation_psi = 0.5

    fig = plt.figure(figsize=(12, 24))   
    for i in trange(len(vec_syn)):
        vec = vec_syn[i].reshape(1,18,512)
        image =  Gs.components.synthesis.run(vec, **Gs_syn_kwargs)        
        PIL.Image.fromarray(image[0], 'RGB')   
        ax = fig.add_subplot(10, 5, i+1, xticks=[], yticks=[])
        image_plt = np.array(image[0])
        ax.imshow(image_plt)
        ax.set_xlabel(str(i), fontsize=20)               
    plt.show()
    plt.close()

# ------------- 潜在変数の指定インデックス間のアニメーション -------
def generate_gif(vec_syn, idx):
    network_pkl = 'gdrive:networks/stylegan2-ffhq-config-f.pkl'  
    
    _G, _D, Gs = pretrained_networks.load_networks(network_pkl)
    noise_vars = [var for name, var in Gs.components.synthesis.vars.items() if name.startswith('noise')]

    Gs_syn_kwargs = dnnlib.EasyDict()
    Gs_syn_kwargs.output_transform = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
    Gs_syn_kwargs.randomize_noise = True 
    Gs_syn_kwargs.truncation_psi = 0.5

    image_gif_256 = []
    os.makedirs('my/gif', exist_ok=True)
    for j in trange(len(idx)-1):
        for i in range(40):
            vec = vec_syn[idx[j]]+(vec_syn[idx[j+1]]-vec_syn[idx[j]])*i/39
            vec = vec.reshape(1, 18, 512)
            images =  Gs.components.synthesis.run(vec, **Gs_syn_kwargs) 
            image_one = PIL.Image.fromarray(images[0], 'RGB')
            image_gif_256.append(image_one.resize((256,256))) 

    print('making_gif...')
    image_gif_256[0].save('./my/gif/anime_256.gif', save_all=True, append_images=image_gif_256[1:],
                      duration=100, loop=0) 

# --------- Style Mixing の実行 -----------
def style_mixing(vec_syn, col_styles, truncation_psi):  
    network_pkl = 'gdrive:networks/stylegan2-ffhq-config-f.pkl'
    row_seeds = [1, 2, 3]
    col_seeds = [4, 5, 6]
    col_styles = col_styles
    truncation_psi = truncation_psi
    minibatch_size = 4

    # Loading networks 
    _G, _D, Gs = pretrained_networks.load_networks(network_pkl)

    Gs_syn_kwargs = dnnlib.EasyDict()
    Gs_syn_kwargs.output_transform = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
    Gs_syn_kwargs.randomize_noise = False
    Gs_syn_kwargs.minibatch_size = minibatch_size

    # Generating W vectors
    all_seeds = list(row_seeds + col_seeds)   
    all_w = vec_syn[:6]
    w_dict = {seed: w for seed, w in zip(all_seeds, list(all_w))} # [layer, component]

    print('Generating images...') 
    all_images = Gs.components.synthesis.run(all_w, **Gs_syn_kwargs) # [minibatch, height, width, channel]
    image_dict = {(seed, seed): image for seed, image in zip(all_seeds, list(all_images))}

    print('Generating style-mixed images...')
    for row_seed in row_seeds:
        for col_seed in col_seeds:
            w = w_dict[row_seed].copy()
            w[col_styles] = w[col_styles] + (w_dict[col_seed][col_styles]-w[col_styles])*truncation_psi
            image = Gs.components.synthesis.run(w[np.newaxis], **Gs_syn_kwargs)[0]
            image_dict[(row_seed, col_seed)] = image

    print('Saving images...')
    os.makedirs('my/stylemix_images', exist_ok=True)
    for (row_seed, col_seed), image in image_dict.items():
        PIL.Image.fromarray(image, 'RGB').save('./my/stylemix_images/'+str(row_seed)+'-'+str(col_seed)+'.png')

    print('Saving image grid...')
    _N, _C, H, W = Gs.output_shape
    canvas = PIL.Image.new('RGB', (W * (len(col_seeds) + 1), H * (len(row_seeds) + 1)), 'black')

    r, c = 4, 4  # スクリーン設定（4行×4列）
    fig, axs = plt.subplots(r, c, figsize=(10,10), subplot_kw=({'xticks':(),'yticks':()}))

    for row_idx, row_seed in enumerate([None] + row_seeds):
        for col_idx, col_seed in enumerate([None] + col_seeds):
            if row_seed is None and col_seed is None:
                continue
            key = (row_seed, col_seed)
            if row_seed is None:
                key = (col_seed, col_seed)
            if col_seed is None:
                key = (row_seed, row_seed)
            canvas.paste(PIL.Image.fromarray(image_dict[key], 'RGB'), (W * col_idx, H * row_idx)) 

            # スクリーンに画像配置            
            image_plt = np.array(image_dict[key])
            axs[row_idx, col_idx].imshow(image_plt)
            if row_seed is None:
                x, y = col_seed, col_seed
            elif col_seed is None:
                x, y = row_seed, row_seed
            else:
                x, y = row_seed, col_seed
            axs[row_idx, col_idx].set_xlabel(str(x)+'-'+str(y))

    canvas.save('./my/stylemix_images/grid.png') 

    # スクリーン表示
    black = np.zeros((1024,1024,3))  # 黒画像作成
    axs[0,0].imshow(black)
    axs[0,0].axis('off')
    plt.show()
    plt.close()   


#2.今回準備した潜在変数を見てみる
sampleデータをロードします。

In [ ]:
vec_direction = np.load('sample/vectors/vec_direction.npy')
vec_smile = np.load('sample/vectors/vec_smile.npy')
vec_glass = np.load('sample/vectors/vec_glass.npy')
vec_young = np.load('sample/vectors/vec_young.npy')
vec_old = np.load('sample/vectors/vec_old.npy')
vec_man = np.load('sample/vectors/vec_man.npy')
vec_all = np.load('sample/vectors/vec_all.npy')

潜在変数vec_allによって生成される画像の一覧を表示します。


In [ ]:
display(vec_all)

モーフィングをやってみます

In [ ]:
from IPython.display import Image
generate_gif(vec_all, [9, 11, 21, 17])
Image('./my/gif/anime_256.gif', format='png')

#3.Style_Mixing
指定された潜在変数wをRow_picとCol_picで入れ替えます。\
第１引数：ベクトル名\
第２引数：Wの指定（リスト形式）\
第３引数：Col_picのWに掛け算する係数(0〜1はRow_colとの混合割合)\
＊生成した画像は、my/stylemix_images に上書き保存します



In [ ]:
# Face_direction
style_mixing(vec_direction, [0,1], 1.4)

In [ ]:
# Smile
style_mixing(vec_smile, [4,5], 1.0)

In [ ]:
# Glasses
style_mixing(vec_glass, [0,1,2], 1.0)

In [ ]:
# UnGlasses
vec_glass1 = np.vstack((vec_glass[3:],vec_glass[:3]))
style_mixing(vec_glass1, [0,1,2], 0.8)

In [ ]:
# Young
style_mixing(vec_young, [4,5,6,7], 0.8)

In [ ]:
# Old
style_mixing(vec_old, [4,5,6,7], 0.6)

In [ ]:
# Man
style_mixing(vec_man, [4,5,6,7], 1.0)